In [ ]:
# Personally I had to add the root folder of the repo to the sys.path.  If certain imports do not work you should uncomment and set the following.
# import sys
# sys.path.append('/root/of/repo/folder/')

# Effects of mobility
In this experiment we evaluate the relationship between the user mobility on the different generators and proposed strategies.  In different runs we evaluate whether our strategies perform better or worse when users move more frequently.  Here again we use the 14 node setup to allow for the most movement.

In [ ]:
from experiments.utils import make_dir, read_node_map

resource_file = "../dataset/out/dataset-resources-stats.csv"
pagemap_file = "../dataset/out/page-map-clean.csv"

node_map_14 = read_node_map('./node_setups/14nodes.json') 
out_dir = make_dir('./out/experiment-mobility-effects/')

belady_out_dir = make_dir(f"{out_dir}beladys/")
lru_out_dir =  make_dir(f"{out_dir}lru/")
cooplru_out_dir = make_dir(f"{out_dir}/cooplru/")
profiles_out_dir = make_dir(f"{out_dir}/profiles/")
federated_out_dir = make_dir(f"{out_dir}/federated/")

## Experiment Configuration
We test 4 different mobility values, `once-every-1000`, `once-every-100`, `once-every-10`, `every-2` iterations.

In [ ]:
no_users = 1000
no_iterations = 5000
no_runs = 10
trace_seeds = [ str(i) for i in range(no_runs) ]
cache_capacity = 1024 * 1024 * 1024

mobility_speeds = [ 0.001, 0.01, 0.1, 0.5 ]

## Traces
We generate traces for the largest node setup as mentioned above.

In [ ]:
from experiments.utils import load_or_generate_trace
from simulation.generator.main_zipf import TraceConfig, Simulation
from simulation.generator.main_page_map import UserTraceConfig, UserSimulation

def generate_zipf_trace(seed: str, zipf_exponent: float, user_mobility: float = 0.05):
    trace_config = TraceConfig(node_map=node_map_14, seed=seed, no_users=no_users, no_iterations=no_iterations, zipf_exponent=zipf_exponent, move_chance=user_mobility)
    simulation = Simulation(trace_config, resource_file)
    return load_or_generate_trace(f"{out_dir}/{trace_config.to_filename()}.trace.gz", simulation=simulation)

def generate_page_map_trace(seed: str, user_mobility: float = 0.05):
    trace_config = UserTraceConfig(node_map=node_map_14, seed=seed, no_users=no_users, no_iterations=no_iterations, move_chance=user_mobility)
    user_simulation = UserSimulation(trace_config, pagemap_file)
    return load_or_generate_trace(f"{out_dir}/{trace_config.to_filename()}.trace.gz", simulation=user_simulation)

## Strategies

In [ ]:
from simulation.evaluator.strategy.belady_min import run_belady
from experiments.utils import read_resource_map

def run_belady_experiment(trace, marker: str = ""):
    run_belady(trace, read_resource_map(resource_file), cache_capacity, belady_out_dir, marker=f"n{len(node_map_14)}-{marker}")

In [ ]:
from experiments.utils import setup_nodes, setup_stats_file_writers, read_resource_map
from simulation.evaluator.strategy.runner import StrategyRunner
from simulation.evaluator.strategy.strategy import CacheStrategy
from simulation.evaluator.strategy.lru import LRUStrategy
from simulation.evaluator.strategy.cooperative_lru import CooperativeLRUStrategy
from simulation.evaluator.strategy.profiles import ProfilesStrategy
from simulation.evaluator.strategy.federated import FederatedStrategy
from typing import Callable

create_lru_setup = lambda nodes: (LRUStrategy(nodes), lru_out_dir)
create_cooplru_setup = lambda nodes: (CooperativeLRUStrategy(nodes, node_trail_length=3), cooplru_out_dir)
create_profiles_setup = lambda nodes: (ProfilesStrategy(nodes, ranking_timeout=5), profiles_out_dir)
create_federated_setup = lambda nodes: (FederatedStrategy(nodes), federated_out_dir)

setups = [ create_lru_setup, create_cooplru_setup, create_profiles_setup, create_federated_setup ]

def run_strategy_experiment(trace, strategy_setup: Callable[[dict[str, dict[str, int]]], CacheStrategy], marker: str = ""):
    nodes = setup_nodes(len(node_map_14), cache_capacity)
    strategy, strat_out_dir = strategy_setup(nodes)    
    stats_writers = setup_stats_file_writers(nodes, strat_out_dir, marker=f"n{len(nodes)}-{marker}")
    StrategyRunner(strategy, trace, read_resource_map(resource_file), stats_writers=stats_writers).perform()

## Experiment
In contrast to previous experiments we need to generate a new trace for every setup as the mobility speed is part of the trace.  This means there is no need to pre-generate the traces, instead, each process can generate their own based on the experiment setup.  We only use the Page Map traces as due to the specialisation of users popularity moves around.

In [ ]:
load_page_map_trace = lambda seed, speed: (generate_page_map_trace(seed=seed, user_mobility=speed), 'page-map')

trace_options = [ load_page_map_trace ]

def run_experiment(trace_seed: str, trace_loader, mobility_speed: float):
    trace, trace_marker = trace_loader(trace_seed, mobility_speed)
    print(trace_seed, trace_marker, mobility_speed)
    run_belady_experiment(trace, marker=f"{trace_marker}-speed-{mobility_speed}-{trace_seed}")
    for setup in setups:
        run_strategy_experiment(trace, setup, marker=f"{trace_marker}-speed-{mobility_speed}-{trace_seed}")
    print(trace_seed, trace_marker, mobility_speed, 'DONE')

In [ ]:
# Make use of multiprocess (over multiprocessing) if an "AttributeError" says it couldn't find `run_experiment`.
from multiprocessing import Pool

if __name__ == '__main__':
    options = [ (seed, trace, speed)
                for seed in trace_seeds 
                for trace in trace_options 
                for speed in mobility_speeds ]
    print(f"Executing {len(options)} experiments...")
    with Pool(4) as p:
        p.starmap(run_experiment, options, chunksize=1)

## Plots

In [ ]:
from experiments.utils import load_runs_in_dir
from palettable.colorbrewer.qualitative import Dark2_8
from palettable.colorbrewer.sequential import Greys_4
dark_8 = Dark2_8.mpl_colors
greys_4 = Greys_4.mpl_colors
import experiments.plotter.neat_plotter

def load_complete_runs(in_dir):
    return [ r for r in load_runs_in_dir(in_dir) if r['iteration'][-1] == 999]

belady_runs = load_runs_in_dir(belady_out_dir)
lru_runs = load_runs_in_dir(lru_out_dir)
cooplru_runs = load_runs_in_dir(cooplru_out_dir)
profiles_runs = load_complete_runs(profiles_out_dir)
federated_runs = load_runs_in_dir(federated_out_dir)
print(len(profiles_runs), len(federated_runs))

In [ ]:
from experiments.utils import calc_ratio, calc_variance

def filter_runs_by(runs, match: str):
    return [ r for r in runs if match in str(r["source"]) ]

def calc_over_setups(runs, strategy: str, calculation) -> list[float]:
    setups = [ f"-speed-{s}-" for s in mobility_speeds ]
    datapoints = []
    for setup in setups:
        filtered_runs = filter_runs_by(filter_runs_by(runs, setup), strategy)
        values = [ calculation(run) for run in filtered_runs ]
        datapoints.append(calc_variance(values))
    return datapoints

calc_average_hit_ratio = lambda run: calc_ratio(run['hits_total'][-1], run['misses_total'][-1])
calc_average_byte_ratio = lambda run: calc_ratio(run['cache_bytes_total'][-1], run['origin_bytes_total'][-1])
calc_average_neighbour_ratio = lambda run: calc_ratio(run['requests_to_neighbours_success'][-1], run['requests_to_neighbours'][-1])
calc_average_neighbour_to_total = lambda run: run['requests_to_neighbours'][-1] / (run['hits_total'][-1] + run['misses_total'][-1])
calc_average_neighbour_bytes = lambda run: run['neighbour_bytes_total'][-1] / (run['cache_bytes_total'][-1] + run['origin_bytes_total'][-1])

In [ ]:
import matplotlib.pyplot as plt
from experiments.utils import generate_comparison_plot
from typing import Tuple
colors = [ greys_4[2] ] + dark_8

def generate_plot(title: str, ylabel, strategies: dict[str, list[Tuple[float, float]]], ylim=(0, 1.0), markers=['.', 'v', 's', 'p', 'P', '*', 'D'], colors=colors, linestyles=['dashed', 'solid', 'solid', 'solid', 'solid']):
    x_labels = [ 1 / speed for speed in mobility_speeds ]
    plt.figure(num=None, figsize=(3, 4), dpi=300)
    generate_comparison_plot(plt, x_labels, strategies, colors=colors, linestyles=linestyles, markers=markers)
    plt.xscale('log')
    plt.ylim(ylim)
    plt.ylabel(ylabel)
    plt.xlabel('Iterations between user movement')
    plt.title(title)
    plt.xticks(x_labels, rotation=60)
    plt.legend()
    plt.show()

In [ ]:
generate_plot(title="Average Hit Ratios Page Map", ylabel='Hit Ratio', strategies={
    "Belady's": calc_over_setups(belady_runs, '-page-map-', calc_average_hit_ratio),
    "LRU": calc_over_setups(lru_runs, '-page-map-', calc_average_hit_ratio),
    "Co-Op LRU": calc_over_setups(cooplru_runs, '-page-map-', calc_average_hit_ratio),
    "Profiles": calc_over_setups(profiles_runs, '-page-map-', calc_average_hit_ratio),
    "Federated": calc_over_setups(federated_runs, '-page-map-', calc_average_hit_ratio)
})

In [ ]:
generate_plot(title="Average Bandwidth Saved Page-Map", ylabel='Fraction of Bandwidth Saved', strategies={
    "Belady's": calc_over_setups(belady_runs, '-page-map-', calc_average_byte_ratio),
    "LRU": calc_over_setups(lru_runs, '-page-map-', calc_average_byte_ratio),
    "Co-Op LRU": calc_over_setups(cooplru_runs, '-page-map-', calc_average_byte_ratio),
    "Profiles": calc_over_setups(profiles_runs, '-page-map-', calc_average_byte_ratio),
    "Federated": calc_over_setups(federated_runs, '-page-map-', calc_average_byte_ratio)
})

### Internal Bandwidth

In [ ]:
from palettable.colorbrewer.diverging import PuOr_4
puor_4 = PuOr_4.mpl_colors

norm_neighbour_success = lambda run: calc_average_neighbour_ratio(run) * calc_average_neighbour_to_total(run)

generate_plot(title="Internal Requests Page Map", ylabel='Internal Request Ratio', strategies={
    "Co-Op LRU": calc_over_setups(cooplru_runs, '-page-map-', calc_average_neighbour_to_total),
    "Co-Op LRU Success": calc_over_setups(cooplru_runs, '-page-map-', norm_neighbour_success),
    "Profiles": calc_over_setups(profiles_runs, '-page-map-', calc_average_neighbour_to_total),
    "Profiles Success": calc_over_setups(profiles_runs, '-page-map-', norm_neighbour_success),
    "Federated": calc_over_setups(federated_runs, '-page-map-', calc_average_neighbour_to_total),
}, markers=['s', 's', 'p', 'p', 'P', 'P'], colors=[ dark_8[1], greys_4[1], dark_8[2], greys_4[2], dark_8[3] ], linestyles=['solid', 'dashed', 'solid', 'dashed', 'solid', 'dashed'] )
print(sum([ avg for avg, std in calc_over_setups(federated_runs, '-page-map-', calc_average_neighbour_to_total) ]) / 4)

In [ ]:
generate_plot(title="Internal Bandwidth Page Map", ylabel='Normalised Average Internal Bandwidth Used', strategies={
    "Co-Op LRU": calc_over_setups(cooplru_runs, '-page-map-', calc_average_neighbour_bytes),
    "Profiles": calc_over_setups(profiles_runs, '-page-map-', calc_average_neighbour_bytes),
    "Federated": calc_over_setups(federated_runs, '-page-map-', calc_average_neighbour_bytes)
}, markers=['s', 'p', 'P'], colors=dark_8[1:], linestyles=['solid'] * 3)
print(sum([ avg for avg, std in calc_over_setups(federated_runs, '-page-map-', calc_average_neighbour_bytes) ]) / 4)